# **VLSI DESIGN USING Z3 SOLVER**

In [ ]:
!pip install z3-solver

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.0/56.0 MB 16.8 MB/s eta 0:00:00


# load and change the input file as needed

In [ ]:
#importing required libraries
import time
start_time = time.time()
import numpy as np
from z3 import And, Or, Int,Not, Solver, sat, If, Sum , Bool,Implies,Optimize

Num = 6 #the number of instance we want to find the solution for.
Inputs = []
#reading the file and seperating each row.
with open( InstanceName , 'r') as Sample:
    for line in Sample:
        Inputs.append(line.replace('\n',''))
#Making a 2 dimensional list of the Inputs in str format.       
Inputs=[line.split(' ') for line in Inputs]
#print(Inputs)

# we must change the inputs from string to integer.
# Defining the inputs and variables. 
Width   = int(Inputs[0][0]) 
Number  = int(Inputs[1][0]) 
Widths  = [int(line[0]) for line in Inputs[2:]] 
Heights = [int(line[1]) for line in Inputs[2:]] 
#defining the upper boundary for the height of plate => calling it MaxHeight.
MaxHeight = sum(Heights) 
#defining the lower boundary for the height of plate => calling it MinHeight.
MinHeight =np.divide((np.sum(np.multiply(Widths,Heights))) , Width)
#Defining the output variables which is the x and y coordinate for each piece.
x_ith =[]
for i in range(Number):
    x_ith.append(Int(f'x{i}') )

y_ith =[]
for i in range(Number):
    y_ith.append(Int(f'y{i}')) 

def FindM(ele):
  Max = ele[0]
  for i in ele[1:]:
    Max = If(i > Max, i, Max)
  return Max    

OptHeight = FindM([y_ith[i] + Heights[i] for i in range(Number)])

#Rotation Variables
Rotation =[]
for i in range(Number):
    Rotation.append(Bool(f'r{i}'))

Widths_r = []
for i in range(Number):
    Widths_r.append(Int(f'w{i}'))

Heights_r = []
for i in range(Number):
    Heights_r.append(Int(f'h{i}'))

#Rotation Constraint
Rot = []
for i in range(Number):
    Rot.append(And(Implies(Rotation[i], Heights_r[i] == Widths[i]), 
    Implies(Rotation[i], Widths_r[i] == Heights[i]),
    Implies(Not(Rotation[i]), Widths_r[i] == Widths[i]),
    Implies(Not(Rotation[i]), Heights_r[i] == Heights[i])))
#Defining the diffn constraints as a function
def diffn(x,y,dx,dy):
    n = len(x)
    overlap = [
        Or(x_ith[i] + Widths[i]  <= x_ith[j],
           x_ith[j] + Widths[j]  <= x_ith[i],
           y_ith[i] + Heights[i] <= y_ith[j],
           y_ith[j] + Heights[j] <= y_ith[i]) 
           for i in range(Number) 
        for j in range(Number) if i < j]
    return overlap
overlap1 = diffn(x_ith,  y_ith, Widths_r, Heights_r)  
sorted_circuits = sorted(zip(Widths, Heights), key=lambda x: x[0]*x[1], reverse=True)
Widths1, Heights1 = zip(*sorted_circuits)  
overlap2 = diffn(x_ith,  y_ith, Widths1, Heights1) 
Bound = [] 
for i in range(Number):
    Bound.append(And(x_ith[i] >= 0, 
        y_ith[i] >= 0,
        x_ith[i] + Widths_r[i] <=Width))  
    
     

In [ ]:
# Creating an optimization object
opt = Optimize()

# Adding constraints
opt.add(overlap1)
opt.add(overlap2)
opt.add(Rot)
opt.add(Bound)
# Adding an optimization goal
# In this case, it's to minimize the height of the plate
opt.minimize(OptHeight)

# Checking if the optimization is satisfiable
if opt.check() == sat:
    model = opt.model()
    # Extracting the values of x and y from the optimization model
    x_vals = [model.eval(x_ith[i]).as_long() for i in range(Number)]
    y_vals = [model.eval(y_ith[i]).as_long() for i in range(Number)]
    print("x_vals: ", x_vals)
    print("y_vals: ", y_vals)
    print("Minimized height: ", model.eval(OptHeight).as_long())
else:
    print("The optimization is not satisfiable")
timing = (time.time() - start_time)
print(timing)    

x_vals:  [0, 4, 10, 7, 7, 10, 3, 0, 0]
y_vals:  [0, 3, 8, 0, 6, 0, 0, 3, 7]
Minimized height:  13
4.657948970794678
